In [6]:
# install GeoPandas before use
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString, Polygon

In [7]:
def generate_OBB(input_path, output_path):
    input_file = gpd.GeoSeries.from_file(input_path)
    gpd.GeoSeries(geom.minimum_rotated_rectangle for geom in input_file).to_file(output_path)

In [8]:
from math import sin, cos, radians, asin, sqrt


def HAV(p1, p2):
    """
    calculate distance through coordinates
    p[0] refers to logitude, p[1] refers to latitude in degrees
    """
    lam1, lam2 = radians(p1[0]), radians(p2[0])
    phi1, phi2 = radians(p1[1]), radians(p2[1])
    arg1 = sin((phi1 - phi2) / 2)
    arg2 = sin((lam1 - lam2) / 2)
    arg3 = cos(phi1) * cos(phi2)

    return 2 * 6371000 * asin(sqrt(arg1 * arg1 + arg3 * arg2 * arg2))


In [9]:
def cut_obb(obb, num1, num2):
    """
    Cut OBB rectangle into num1*num2 grids
    obb -> geom.minimum_rotated_rectangle.exterior.coords
    num1 applies to p1-p2 edge, num2 applies to p2-p3 edge
              num1
    p4 +----------------+ p3
       |                |
       |                | num2
       |                |
    p1 +----------------+ P2
    """
    p1, p2, p3, p4 = Point(obb[0]), Point(obb[1]), Point(obb[2]), Point(obb[3])
    line1, line2 = LineString([p1, p2]), LineString([p4, p3])

    points = []
    # Generate grid points
    for i in range(num1 + 1):
        pointA = line1.interpolate(i / num1, normalized=True)
        pointB = line2.interpolate(i / num1, normalized=True)
        curr_line = LineString([pointA, pointB])
        curr_row = [curr_line.interpolate(i / num2, normalized=True) for i in range(num2 + 1)]
        points.append(curr_row)

    polygons = []
    # Connect points to create grids
    for i in range(num1):
        row1, row2 = points[i], points[i + 1]
        polygons.extend(Polygon([row1[j], row2[j], row2[j + 1], row1[j + 1]]) for j in range(num2))

    return polygons


In [15]:
input_path = './data/15_label.shp'
interval = 1  # 1 meter
half_interval = interval * 0.5

input_file = gpd.GeoSeries.from_file(input_path)
generate_OBB(input_path, './data/15_label_obb.geojson')

lst = []
for geom in input_file:
    obb = geom.minimum_rotated_rectangle.exterior
    p1, p2, p3, p4, _ = list(obb.coords)
    edge1, edge2 = HAV(p1, p2), HAV(p2, p3)

    cut_num_1 = round(edge1 / interval) + int(edge1 < half_interval)
    cut_num_2 = round(edge2 / interval) + int(edge2 < half_interval)

    lst.append(gpd.GeoSeries(cut_obb(obb.coords, cut_num_1, cut_num_2)))

recs = pd.concat(lst)
recs.to_file(f'./data/cut_rec_{interval}m.geojson')
recs.centroid.to_file(f'./data/cut_rec_centroid_{interval}m.geojson')